First Program with Langchain tools 

1) Building a simple agent with Langchain
2) Create a mathematical Toolkit that allows an AI agent to perform basic arithmetic operations through natural language interaction
3) How agent can understand and solve mathematical queries  breaking down the complex operations into simpler steps




# Table of Contents

1. [Objectives](#Objectives)
2. [Setup](#Setup)
3. [Installing required libraries](#Installing-required-libraries)
4. [Loading the LLM: Choosing the right language model](#Loading-the-LLM:-Choosing-the-right-language-model)
5. [Function](#Function)
   - 5.1 [Tool](#Tool)
   - 5.2 [initialize_agent](#initialize_agent)
6. [Relationship between agent and LLM](#Relationship-between-agent-and-LLM)
7. [Key parameters of initialize_agent](#Key-parameters-of-initialize_agent)
8. [Orchestrating multiple tools with an agent: Mathematical toolkit](#Orchestrating-multiple-tools-with-an-agent:-Mathematical-toolkit)
   - 8.1 [Subtraction tool](#Subtraction-tool)
9. [Building the agent](#Building-the-agent)
   - 9.1 [Exploring LangChain's built-in tools](#Exploring-LangChain's-built-in-tools)
   - 9.2 [Popular built-in tools](#Popular-built-in-tools)
   - 9.3 [Example: Using the Wikipedia tool](#Example:-Using-the-Wikipedia-tool)
10. [Exercise: Create a power tool to calculate exponents](#Exercise:-Create-a-power-tool-to-calculate-exponents)
    - 10.1 [Objective](#Objective)
    - 10.2 [Step 1: Create the power tool](#Step-1:-Create-the-power-tool)
    - 10.3 [Step 2: Create an agent with the power tool](#Step-2:-Create-an-agent-with-the-power-tool)
    - 10.4 [Step 3: Test the agent](#Step-3:-Test-the-agent)
11. [Authors](#authors)


## Objectives

After completing this lab, you will be able to:

- Explain the concept of tools in LangChain
- Create custom tools for specific tasks
- Build an AI agent that can use multiple tools
- Debug and improve tool functionality
- Test tool implementations with various inputs


----


## Setup



For this lab, you will use the following libraries:

- **`langchain`**: For creating AI agents and tools
- **`langchain.chat_models`**: For accessing language models
- **`langchain.agents`**: For creating and managing AI agents

---

## Installing required libraries

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
%pip install langchain==0.3.23 | tail -n 1 
%pip install langchain-ibm==0.3.10 | tail -n 1 
%pip install langchain-community==0.3.16 | tail -n 1 
%pip install wikipedia==1.4.0 | tail -n 1
%pip install openai==1.77.0 | tail -n 1
%pip install langchain-openai==0.3.16 | tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import the required libraries


In [2]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType
import re

## Loading the LLM: Choosing the right language model

In this example, IBM’s `ChatWatsonx`will be used to load a language model (LLM) for interacting with tools. IBM’s models, like Granite 3.2 and Granite 3.3, are highly versatile and excel at advanced reasoning tasks.

That said, other providers offer LLMs with different strengths:

- **OpenAI (GPT-4/GPT-3.5)**: Best for versatility and advanced reasoning.
- **Facebook (Meta, LLaMA)**: Open-access, highly customizable for specialized use cases.
- **IBM WatsonX Granite**: Ideal for enterprise applications with seamless integration.
- **Anthropic (Claude)**: Focused on safety, reliability, and ethical AI.
- **Cohere**: Affordable and efficient for lightweight, task-specific models.

---

For this project, you'll use `ChatWatsonx` because:
- It offers a simple API for quick setup.
- It supports advanced configurations like:
  - **`temperature`**: Adjusting randomness of responses.
  - **`max_tokens`**: Limiting the length of responses.
- IBM’s models are widely regarded as state-of-the-art for general-purpose reasoning and conversation.


In [3]:


llm = ChatWatsonx(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
)

Let's generate a simple response:


In [7]:
response = llm.invoke("What is tool calling in langchain ?")
#print("Response 1:", response)
print("\nResponse Content: ", response.content)




Response Content:  In the context of Langchain, a project focused on developing language models and tools for AI research, "tool calling" generally refers to the mechanism by which external tools or services are invoked or integrated within the framework of their language models or applications.

Here's a more detailed explanation:

1. **Integration of External Tools**: Langchain's models are not static; they are designed to interface with a wide array of external software tools, databases, or APIs. This allows the models to perform tasks that go beyond basic language processing, such as data analysis, web scraping, or manipulating files.

2. **Dynamic Functionality**: Tool calling enables the dynamic extension of the model's capabilities. For instance, a language model in Langchain might be programmed to call a specific API for real-time weather information, a database for factual data, or a translation service for multi-lingual support.

3. **Automated Workflows**: Tool calling is a

## This verifies that LLM is working fine



## Function

In AI, a **tool** will call a basic **function** or capability that can be called on to perform a specific task. Think of it like a single item in a toolbox: just like a hammer, screwdriver, or wrench, an AI toolbox is full of specific functions designed to solve problems or get things done.

When building tools for tool calling, there are a few key principles to keep in mind:

1. **Clear purpose**: Make sure the tool has a well-defined job.
2. **Standardized input**: The tool should accept input in a predictable, structured format so it’s easy to use.
3. **Consistent output**: Always return results in a format that’s easy to process or integrate with other systems.
4. **Comprehensive documentation**: Your tool should include clear, simple documentation that explains what it does, how to use it, and any quirks or limitations.

Remember, documentation isn’t just for other developers—it’s also for the language model (LLM) to understand the tool’s purpose and how to use it effectively.

For this example, you’ll start with a simple tool to add numbers. It’ll check off most of the basic requirements, but one key limitation is that it doesn’t handle **basic error cases**, like ignoring non-numeric input. Improving error handling will make the tool much more robust and ready for real-world use.


In [5]:
def add_numbers(inputs:str) -> dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string.

    Parameters:
    - inputs (str): 
    string, it should contain numbers that can be extracted and summed.

    Returns:
    - dict: A dictionary with a single key "result" containing the sum of the numbers.

    Example Input (Dictionary):
    {"numbers": [10, 20, 30]}

    Example Input (String):
    "Add the numbers 10, 20, and 30."

    Example Output:
    {"result": 60}
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]

    
    result = sum(numbers)
    return {"result": result}

## Defines a function to add numbers extracted from a string input 

In [8]:
add_numbers("1 2") 

{'result': 3}


## Tool
The `Tool` class in LangChain serves as a structured wrapper that converts regular Python functions into agent-compatible tools. Each tool needs three key components:
1. A name that identifies the tool
2. The function that performs the actual operation
3. A description that helps the agent understand when to use the tool

Testing section improvement:




In [9]:
from langchain.agents import  Tool
add_tool=Tool(
        name="AddTool",
        func=add_numbers,
        description="Adds a list of numbers and returns the result.")

print("tool object",add_tool)

tool object name='AddTool' description='Adds a list of numbers and returns the result.' func=<function add_numbers at 0x7842ea1c85e0>


# Purpose wraps the add_number function into a langchain Tool object

# The Tool class provides a structured interface for the agent to call the function, with metadata to guide its usage.

Let's see the parameters of the object:

- **`name`** (*str*):
  - A unique identifier for the tool.

  - **Example**: `"AddTool"`

- **`.invoke`** (*Callable*):
  - The function that the tool wraps.
  - **Example**: `add_numbers`

- **`description`** (*str*):
  - A concise explanation of what the tool does.
  - **Example**: `"Adds a list of numbers and returns the result."`




These attributes allow you to inspect the tool object


In [11]:
# Tool name
print("Tool Name:")
print(add_tool.name)

# Tool description
print("Tool Description:")
print(add_tool.description)

# Tool function
print("Tool Function:")
print(add_tool.invoke)


Tool Name:
AddTool
Tool Description:
Adds a list of numbers and returns the result.
Tool Function:
<bound method BaseTool.invoke of Tool(name='AddTool', description='Adds a list of numbers and returns the result.', func=<function add_numbers at 0x7842ea1c85e0>)>


You can call the tool's function via the ```add_tool``` object:


In [12]:
print("Calling Tool Function:")
test_input = "10 20 30 a b" 
print(add_tool.invoke(test_input))  # Example

Calling Tool Function:
{'result': 60}


Testing the tool object ensures:

1. **The tool is correctly set up**:
   - Metadata (`name`, `description`, etc.) is properly defined and aligns with its purpose.
   - The function and schema (if applicable) are correctly configured.

2. **The wrapped function behaves as expected**:
   - The function performs the intended task correctly.
   - It handles edge cases and invalid inputs gracefully.

3. **The tool integrates smoothly with agents**:
   - The tool's output aligns with what the agent expects.
   - There are no compatibility issues when the agent calls the tool.


# the @tool operator is a Python decorator used to define custom tools that an agent can call during execution.

### `@tool` operator

Now you know how to create a tool with a `Tool` class (using Tool Interface), there's actually another way that you can create a tool using `@tool` decorator. The recommended way to create tools is using the `@tool` decorator. This decorator is designed to simplify the process of tool creation and should be used in most cases. After defining a function, you can decorate it with `@tool` to create a tool that implements the Tool Interface.

`@tool` opertor makes tools out of functions. See below:


In [13]:
from langchain_core.tools import tool
import re

@tool
def add_numbers(inputs:str) -> dict:
    """
    Adds a list of numbers provided in the input string.
    Parameters:
    - inputs (str): 
    string, it should contain numbers that can be extracted and summed.
    Returns:
    - dict: A dictionary with a single key "result" containing the sum of the numbers.
    Example Input:
    "Add the numbers 10, 20, and 30."
    Example Output:
    {"result": 60}
    """
    # Use regular expressions to extract all numbers from the input
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    # numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    
    result = sum(numbers)
    return {"result": result}

The @tool decorator automatically creates a StructuredTool with schema information derived from the function’s signature and docstring.

In [14]:
print("Name: \n", add_numbers.name)
print("Description: \n", add_numbers.description) 
print("Args: \n", add_numbers.args) 


Name: 
 add_numbers
Description: 
 Adds a list of numbers provided in the input string.
Parameters:
- inputs (str): 
string, it should contain numbers that can be extracted and summed.
Returns:
- dict: A dictionary with a single key "result" containing the sum of the numbers.
Example Input:
"Add the numbers 10, 20, and 30."
Example Output:
{"result": 60}
Args: 
 {'inputs': {'title': 'Inputs', 'type': 'string'}}


You can call the tool using the ```invoke``` method.


In [15]:
test_input = "what is the sum between 10, 20 and 30 " 
print(add_numbers.invoke(test_input)) 

{'result': 60}



### Use @tool-StructuredTool

The @tool decorator creates a StructuredTool with schema information extracted from function signatures and docstrings as show here. This helps LLMs better understand what inputs the tool expects and how to use it properly. While both approaches work, @tool is generally preferred for modern LangChain applications, especially with LangGraph and function-calling models.


In [16]:
# Comparing the two approaches
print("Tool Constructor Approach:")

print(f"Has Schema: {hasattr(add_tool, 'args_schema')}")
print("\n")

print("@tool Decorator Approach:")


print(f"Has Schema: {hasattr(add_numbers, 'args_schema')}")
print(f"Args Schema Info: {add_numbers.args}")

Tool Constructor Approach:
Has Schema: True


@tool Decorator Approach:
Has Schema: True
Args Schema Info: {'inputs': {'title': 'Inputs', 'type': 'string'}}


In this example, the tool has two inputs: a string containing the numbers to add, and a second boolean input that determines whether to sum the absolute values of those numbers.
